In [1]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from datetime import datetime
import os
import io

# Connect to Azure Blob Storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=sg092620240215;AccountKey=+PaTF6WCZ0NY63Hni1XIWRJfWsnTI7QJCLVP0f1OXUoVzJyl0AcE4h2Pe1b7ZbgldGkDDFA0j9iK+AStvU4auA==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
silver_client = blob_service_client.get_container_client("silver")

# Find latest all_players_*.csv
all_players_blob = None
latest_time = None
blob_list = silver_client.list_blobs(name_starts_with="all_players_")
print("Checking for all_players_ files:")
for blob in blob_list:
    print(f"Found: {blob.name}")
    try:
        parts = blob.name.split("_")
        if len(parts) < 3:
            print(f"Skipping {blob.name}: Not enough parts in filename")
            continue
        timestamp_str = f"{parts[-2]}_{parts[-1].replace('.csv', '')}"
        timestamp = datetime.strptime(timestamp_str, "%Y%m%d_%H%M%S")
        if latest_time is None or timestamp > latest_time:
            latest_time = timestamp
            all_players_blob = blob.name
    except ValueError as e:
        print(f"Skipping {blob.name}: Invalid timestamp format ({str(e)})")
        continue

if not all_players_blob:
    raise ValueError("No valid all_players_*.csv files found in silver")

# Hardcode flattened_rosters.csv
rosters_blob = "flattened_rosters.csv"
print(f"Checking for {rosters_blob}:")
if not silver_client.get_blob_client(rosters_blob).exists():
    raise ValueError(f"{rosters_blob} not found in silver")

# Load all_players_*.csv
print(f"Loading {all_players_blob}")
all_players_client = silver_client.get_blob_client(all_players_blob)
all_players_data = all_players_client.download_blob().readall().decode("utf-8")
df_players = pd.read_csv(io.StringIO(all_players_data))

# Load flattened_rosters.csv
print(f"Loading {rosters_blob}")
rosters_client = silver_client.get_blob_client(rosters_blob)
rosters_data = rosters_client.download_blob().readall().decode("utf-8")
df_rosters = pd.read_csv(io.StringIO(rosters_data))

# Merge on 'id'
merged_df = pd.merge(df_players, df_rosters, on="id", how="inner")
if merged_df.empty:
    raise ValueError("No matching IDs found between all_players and flattened_rosters")

# Transformations
# 1. Adjust salary: if < 1000, multiply by 1,000,000
merged_df.loc[merged_df['salary'] < 1000, 'salary'] *= 1000000

# 2. Add salary_display column
merged_df['salary_display'] = merged_df['salary'].apply(lambda x: f"${int(x / 1000000)}M")

# 3. Drop the 'status' column
merged_df = merged_df.drop(columns=['status'])

# Save merged CSV to silver
output_blob_name = f"merged_rosters_players_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
output_blob_client = silver_client.get_blob_client(output_blob_name)
merged_csv = merged_df.to_csv(index=False)
output_blob_client.upload_blob(merged_csv, overwrite=True)
print(f"Uploaded {output_blob_name} to silver container")

StatementMeta(, , -1, Waiting, , Waiting)